In [3]:
from ultralytics import YOLO

# 1. Обучение модели

Беру предобученную модель на датасете COCO

In [ ]:
# model = YOLO('/container_root/ws/models/yolo11n-seg.pt')
# seg_res = model(path_to_image)

# 2. Профилирование модели PyTorch

In [4]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [14]:
model = YOLO('/container_root/ws/models/yolo11n-seg.pt')
img_path = '/container_root/ws/notebooks/bus.jpg'

## Анализ времени выполнения

In [20]:

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(img_path)

print(prof.key_averages(group_by_input_shape=True).table(sort_by="self_cpu_time_total", row_limit=100))


image 1/1 /container_root/ws/notebooks/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 189.0ms
Speed: 4.7ms preprocess, 189.0ms inference, 8.4ms postprocess per image at shape (1, 3, 640, 480)
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference        20.59%      47.553ms       100.00%     230.948ms     230.948ms             1                                                                                [] 

## Анализ потребления памяти

In [23]:
with profile(activities=[ProfilerActivity.CPU], profile_memory=True, record_shapes=True) as prof:
    with record_function("model_inference"):
        model(img_path)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=100))


image 1/1 /container_root/ws/notebooks/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 177.1ms
Speed: 4.3ms preprocess, 177.1ms inference, 6.6ms postprocess per image at shape (1, 3, 640, 480)
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.76%       1.573ms         0.76%       1.573ms       6.666us      78.86 Mb      78.86 Mb           236  
                        aten::cat         3.82%       7.864ms         4.12%       8.500ms     326.904us      32.38 Mb      32.38 Mb            26  
        aten::upsample_bilinear2d         1.40%       2.878ms     